In [1]:
import pandas as pd

In [ ]:
# Load the dataset
df = pd.read_csv('Ranking_History_studylink.govt.nz_12_27_2024_all.csv')

In [ ]:
# Observe the dataset
df.head()

In [ ]:
df.info()

In [ ]:
# Drop specified columns
df.drop(columns=['TopLevel', 'SecondLevel', 'ThirdLevel', 'FourthLevel', 'FifthLevel'], inplace=True)

In [ ]:
# Fill NaN values in numeric columns with 0
numeric_columns_to_fill = ['StartRank', 'EndRank', 'RankChange', 'EndClicks', 'ClicksChange', 'SearchVolume']
df[numeric_columns_to_fill] = df[numeric_columns_to_fill].fillna(0)

In [ ]:
# Identify missing values in all columns
missing_values = df.isnull().sum()

# Display missing values
print(missing_values)

# Display the first few rows of the cleaned dataset
df.head()

In [ ]:
# Ignore missing values in the dates columns

In [ ]:
# Check for duplicate values in 'Keyword' column
duplicates = df['Keyword'].duplicated().sum()
print(f'Number of duplicate Keywords: {duplicates}')

In [ ]:
# Remove duplicate rows based on 'Keyword', keeping the first occurrence
df = df.drop_duplicates(subset='Keyword', keep='first')

In [ ]:
# Drop more columns which are not revlevance to our analysis
drop_columns = ['StartRank', 'RankChange', 'ClicksChange']
df = df.drop(columns=drop_columns)

In [ ]:
df.info()

In [ ]:
# Open the dataset from webscraping, to get the latest meta description data

In [ ]:
df2=pd.read_csv('StudyLink_DateConverted.csv')

In [ ]:
df2.info()

In [ ]:
# Display column names to verify the correct name for 'Timestamp'
print(df2.columns)

In [ ]:
# Ensure the correct column name is used for datetime conversion
# Replace 'Timestamp' with the correct column name from the printed columns
correct_timestamp_column = 'Timestamp'  # Update this if necessary
df2[correct_timestamp_column] = pd.to_datetime(df2[correct_timestamp_column], errors='coerce')

In [ ]:
# Find the row with the latest Timestamp
latest_row = df2.loc[df2['Timestamp'].idxmax()]


In [ ]:
# Get the value in the 'keywords' column for the latest Timestamp
latest_description = latest_row['description']

In [ ]:
print("Latest description:", latest_description)

In [ ]:
# Add a new column 'description' to the first dataset 

In [ ]:
# Add a new column 'description'
df['description'] = latest_description

In [ ]:
df.head()

In [ ]:
# Perform TF-IDF analysis on 'Keyword' and 'description'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
# Perform TF-IDF analysis on 'Keyword' and 'description'
df['Keyword'] = df['Keyword'].fillna('')
df['description'] = df['description'].fillna('')

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Keyword'] + ' ' + df['description'])
df['TF-IDF_score'] = np.mean(tfidf_matrix.toarray(), axis=1)

In [ ]:
df.head()

In [ ]:
df.to_csv('rank_keywords_studylink_cleaned_score.csv', index=False)

In [ ]:
# Impute on the date data 

In [ ]:
import pandas as pd

In [15]:
df=pd.read_csv('rank_keywords_studylink_cleaned_score.csv')

In [ ]:
# Create a subset 'df_rank_change' by dropping specified columns
columns_to_drop = ['EndRank', 'EndClicks', 'SearchVolume', 'description', 'TF-IDF_score']
df_rank_change = df.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
# Display the first few rows of the subset to verify
df_rank_change.head()

In [ ]:
# Transpose the dataframe, making the rows become columns
df_rank_change = df_rank_change.transpose().reset_index()

In [ ]:
df_rank_change.head()

In [ ]:
df_rank_change.columns = df_rank_change.iloc[0]

In [ ]:
# Drop the first row of the dataframe
df_rank_change = df_rank_change.iloc[1:].reset_index(drop=True)

In [ ]:
df_rank_change.head()

In [ ]:
# Rename the first row's 'Keyword' column to 'Date'
df_rank_change.rename(columns={'Keyword': 'Date'}, inplace=True)

In [ ]:
df_rank_change.head()

In [ ]:
# Convert 'Date' column to datetime with format 'mm/yyyy'
df_rank_change['Date'] = pd.to_datetime(df_rank_change['Date'], format='%m/%Y', errors='coerce').dt.strftime('%m/%Y')

In [ ]:
# Ensure target columns are numeric
target_cols = df_rank_change.columns[1:]
df_rank_change[target_cols] = df_rank_change[target_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Impute missing values in numeric columns using KNearestNeighbors
from sklearn.impute import KNNImputer
import numpy as np

In [ ]:
imputer = KNNImputer(n_neighbors=5)
numeric_cols = df_rank_change.select_dtypes(include=np.number).columns
if not numeric_cols.empty:
    df_rank_change[numeric_cols] = imputer.fit_transform(df_rank_change[numeric_cols])

In [ ]:
df_rank_change.head()

In [ ]:
# Store the transposed data into a csv for more imputation later on
df_rank_change.to_csv('rank_keywords_studylink_cleaned_transposed.csv', index=False)

In [ ]:
###

In [3]:
# Import the transposed dataset again
df_rank_change = pd.read_csv('rank_keywords_studylink_cleaned_transposed.csv')

In [5]:
# Transpose the dataframe back to original shape
df_rank_change = df_rank_change.transpose()

In [7]:
df_rank_change.columns = df_rank_change.iloc[0]

In [11]:
df_rank_change = df_rank_change.iloc[1:].reset_index(drop=True)

In [13]:
df_rank_change.head()

Date,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021,11/2021,12/2021,...,03/2024,04/2024,05/2024,06/2024,07/2024,08/2024,09/2024,10/2024,11/2024,12/2024
0,17.333333,17.4,18.0,16.2,17.6,18.0,17.6,18.0,17.4,18.0,...,14.0,18.0,18.0,17.2,16.0,18.0,17.6,17.6,17.0,17.4
1,38.909091,41.4,37.6,37.8,42.0,39.8,42.0,42.0,42.0,40.0,...,38.6,36.4,39.0,37.8,35.0,38.8,36.0,41.6,35.0,39.0
2,74.285714,68.8,72.8,71.0,72.8,68.8,68.8,68.8,72.8,68.8,...,68.8,91.0,75.6,79.6,79.6,79.6,73.8,80.0,74.0,79.0
3,64.666667,62.8,62.8,66.6,62.8,62.8,62.8,62.8,62.8,62.8,...,62.8,55.0,60.8,84.0,67.4,67.4,62.8,74.0,67.4,62.8
4,69.444444,70.2,70.6,69.0,69.8,68.4,70.2,70.6,68.8,69.8,...,68.0,66.0,69.8,70.0,69.8,77.0,70.6,66.0,69.4,68.6


In [17]:
# Create a new dataframe 'df_imputed' by joining selected columns from 'df' to 'df_rank_change'
selected_columns = ['Keyword', 'EndRank', 'EndClicks', 'SearchVolume', 'description', 'TF-IDF_score']
df_imputed = pd.concat([df[selected_columns], df_rank_change], axis=1)

In [19]:
df_imputed.head()

,Keyword,EndRank,EndClicks,SearchVolume,description,TF-IDF_score,03/2021,04/2021,05/2021,06/2021,...,03/2024,04/2024,05/2024,06/2024,07/2024,08/2024,09/2024,10/2024,11/2024,12/2024
0,academic,17,12,1200.0,We help students make informed choices about t...,0.001933,17.333333,17.4,18.0,16.2,...,14.0,18.0,18.0,17.2,16.0,18.0,17.6,17.6,17.0,17.4
1,otago academic dates,35,0,0.0,We help students make informed choices about t...,0.002098,38.909091,41.4,37.6,37.8,...,38.6,36.4,39.0,37.8,35.0,38.8,36.0,41.6,35.0,39.0
2,university of auckland academic calendar,79,0,0.0,We help students make informed choices about t...,0.002211,74.285714,68.8,72.8,71.0,...,68.8,91.0,75.6,79.6,79.6,79.6,73.8,80.0,74.0,79.0
3,auckland university academic calendar,74,0,0.0,We help students make informed choices about t...,0.002110,64.666667,62.8,62.8,66.6,...,62.8,55.0,60.8,84.0,67.4,67.4,62.8,74.0,67.4,62.8
4,academic history vuw,66,0,0.0,We help students make informed choices about t...,0.001977,69.444444,70.2,70.6,69.0,...,68.0,66.0,69.8,70.0,69.8,77.0,70.6,66.0,69.4,68.6


In [25]:
# Store the imputed data to a new dataset, ready for import into dashboard
df_imputed.to_csv('rank_keywords_studylink_cleaned_imputed.csv', index=False)